In [2]:
import os

In [3]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Diet_Recommendation'

In [6]:
os.chdir("..")

In [7]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

## Update the Enity

In [8]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir : Path
    

In [9]:
from Healthyai.constants import *
from Healthyai.utils.common import read_yaml,create_directories

## Configuration Manager in Src Config

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        
        create_directories([self.config.artifacts])
        
        
        #data ingestion related configuration
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.diet_recommendation_data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config  = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file  = config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config    


## Update the components

In [11]:
import os
import zipfile
import gdown
from Healthyai import logger
from Healthyai.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    def download_data(self) -> str:
        """
        Fetch data from the url
        """
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/diet_recomendation/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        
        """
        Zip_file_path: str
        Extract the zip file into the data directory
        Function retruns None
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
            
            

## Update the pipeline

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2023-12-05 17:42:20,385: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 17:42:20,386: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 17:42:20,387: INFO: common: Created directory at : artifacts]
[2023-12-05 17:42:20,388: INFO: common: Created directory at : artifacts/diet_recomendation/data_ingestion]
[2023-12-05 17:42:20,388: INFO: 1523347044: Downloading data from https://drive.google.com/file/d/1dOGy_vMtQU3YqETamp2CNo78w8-ckLWN/view?usp=drive_link into file artifacts/diet_recomendation/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1dOGy_vMtQU3YqETamp2CNo78w8-ckLWN
To: d:\Nikku\End to End Project\Healthy.ai\artifacts\diet_recomendation\data_ingestion\data.zip
100%|██████████| 29.7k/29.7k [00:00<00:00, 943kB/s]

[2023-12-05 17:42:22,705: INFO: 1523347044: Downloaded data from https://drive.google.com/file/d/1dOGy_vMtQU3YqETamp2CNo78w8-ckLWN/view?usp=drive_link into file artifacts/diet_recomendation/data_ingestion/data.zip]
